<a href="https://colab.research.google.com/github/hsnam95/class2023Spring/blob/main/gpt_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
import openai
import json
openai.api_key = 'openai key here'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.2 MB/s eta 0:00:00


In [3]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_order(item, size, number, option):
    order_info = {
        "item": item,
        "size": size,
        "number": number,
        "option": option
    }
    return json.dumps(order_info)

In [20]:
# Step 1: send the conversation and available functions to GPT
message = "I'd like to order two large size iced americao"
messages = [{"role": "user", "content": message}]
functions = [
    {
        "name": "get_order",
        "description": "Get the order info in a given conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "item": {"type": "string", "description": "the drink type, e.g. iced latte"},
                "size": {"type": "string", "description": "the drink size, e.g. large"},
                "number": {"type": "string", "description": "the number of orders, e.g. two"},
                "option": {"type": "string", "description": "the drink option, e.g. whipped cream"},
            },
            "required": ["item"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response
response_message = response["choices"][0]["message"]

In [21]:
response_message

<OpenAIObject at 0x7fd9979b89a0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_order",
    "arguments": "{\n  \"item\": \"iced americano\",\n  \"size\": \"large\",\n  \"number\": \"two\"\n}"
  }
}

In [ ]:
# Step 2: check if GPT wanted to call a function
if response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_order": get_order,
    }  # only one function in this example, but you can have multiple
    function_name = response_message["function_call"]["name"]
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = fuction_to_call(
        item=function_args.get("item"),
        size=function_args.get("size"),
        number=function_args.get("number"),
        option=function_args.get("option"),
    )

    # Step 4: send the info on the function call and function response to GPT
    messages.append(response_message)  # extend conversation with assistant's reply
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
    )  # get a new response from GPT where it can see the function response
second_response

In [23]:
response_message.get("function_call")

<OpenAIObject at 0x7fd99788e6b0> JSON: {
  "name": "get_order",
  "arguments": "{\n  \"item\": \"iced americano\",\n  \"size\": \"large\",\n  \"number\": \"two\"\n}"
}